In [101]:
import pandas as pd
import sqlite3
import psycopg

In [102]:
providers_df = pd.read_csv('providers_data.csv')
receivers_df = pd.read_csv('receivers_data.csv')
food_listings_df = pd.read_csv('food_listings_data.csv')
claims_df = pd.read_csv('claims_data.csv')

In [103]:
providers_df

,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577
...,...,...,...,...,...,...
995,996,"Vasquez, Ruiz and Flowers",Restaurant,"84308 Justin Stravenue\nNew Amberside, NE 53447",Williamview,+1-319-378-7627x0682
996,997,Garza-Williams,Catering Service,"08864 Figueroa Radial Suite 948\nJennaberg, AZ...",East Rossside,001-924-441-3963x746
997,998,Novak Group,Grocery Store,"934 Zachary Run\nMelissamouth, WY 02729",Joshuastad,(903)642-1969x3300
998,999,Moody Ltd,Grocery Store,"17580 Ernest Hills\nLake Michaelmouth, OR 56416",Stevenchester,637.300.3664x4880


In [104]:
receivers_df

,Receiver_ID,Name,Type,City,Contact
0,1,Donald Gomez,Shelter,Port Carlburgh,(955)922-5295
1,2,Laurie Ramos,Individual,Lewisburgh,761.042.1570
2,3,Ashley Mckee,NGO,South Randalltown,691-023-0094x856
3,4,Erika Rose,NGO,South Shaneville,8296491111
4,5,John Romero,Individual,Bakerport,067.491.0154
...,...,...,...,...,...
995,996,Matthew Curtis,Shelter,Lanechester,127-889-4442x1289
996,997,Amanda Cain,NGO,New Steven,+1-001-491-5601x5316
997,998,Theodore Briggs,Individual,South Sandra,930-609-9442x5031
998,999,Cheyenne Ramsey,NGO,Lake Jeffery,001-326-320-4816x15300


In [105]:
food_listings_df

,Food_ID,Food_Name,Quantity,Expiry_Date,Provider_ID,Provider_Type,Location,Food_Type,Meal_Type
0,1,Bread,43,3/17/2025,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast
1,2,Soup,22,3/24/2025,791,Grocery Store,West James,Non-Vegetarian,Dinner
2,3,Fruits,46,3/28/2025,478,Catering Service,Lake Regina,Vegan,Breakfast
3,4,Fruits,15,3/16/2025,930,Restaurant,Kellytown,Vegan,Lunch
4,5,Soup,14,3/19/2025,279,Restaurant,Garciaport,Vegan,Dinner
...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,3/30/2025,467,Catering Service,Phillipsfort,Vegan,Breakfast
996,997,Fish,22,3/18/2025,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast
997,998,Fruits,6,3/22/2025,444,Restaurant,New Billy,Non-Vegetarian,Dinner
998,999,Pasta,15,3/30/2025,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch


In [106]:
claims_df

,Claim_ID,Food_ID,Receiver_ID,Status,Timestamp
0,1,164,908,Pending,3/5/2025 5:26
1,2,353,391,Cancelled,3/11/2025 10:24
2,3,626,492,Completed,3/21/2025 0:59
3,4,61,933,Cancelled,3/4/2025 9:08
4,5,345,229,Pending,3/14/2025 15:17
...,...,...,...,...,...
995,996,855,211,Completed,3/13/2025 19:40
996,997,980,746,Cancelled,3/17/2025 22:04
997,998,832,967,Cancelled,3/13/2025 18:00
998,999,917,90,Completed,3/1/2025 15:31


In [107]:
print(food_listings_df.info())
print(food_listings_df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Food_ID        1000 non-null   int64 
 1   Food_Name      1000 non-null   object
 2   Quantity       1000 non-null   int64 
 3   Expiry_Date    1000 non-null   object
 4   Provider_ID    1000 non-null   int64 
 5   Provider_Type  1000 non-null   object
 6   Location       1000 non-null   object
 7   Food_Type      1000 non-null   object
 8   Meal_Type      1000 non-null   object
dtypes: int64(3), object(6)
memory usage: 70.4+ KB
None
   Food_ID Food_Name  Quantity Expiry_Date  Provider_ID     Provider_Type  \
0        1     Bread        43   3/17/2025          110     Grocery Store   
1        2      Soup        22   3/24/2025          791     Grocery Store   
2        3    Fruits        46   3/28/2025          478  Catering Service   
3        4    Fruits        15   3/16/2025          930   

In [108]:
!pip install psycopg
import psycopg
!pip install psycopg2
import psycopg2

In [109]:
DB_CONFIG = {
    "dbname" == "ramji",
    "username" == "postgres",
    "password" == "Ktm@9291",
    "host" == "localhost",
    "port" == 5432
}

connection = psycopg2.connect(host = "localhost",database = "ramji", user = "postgres" , password ="Ktm@9291")
cur = connection.cursor()


In [110]:
# Create providers table
cur.execute("""
CREATE TABLE IF NOT EXISTS providers (
Provider_ID INTEGER PRIMARY KEY,
Name TEXT,
Type TEXT,
Address TEXT,
City TEXT,
Contact TEXT
);
""")
connection.commit()
connection.autocommit = True

In [111]:
# Create receivers table
cur.execute("""
CREATE TABLE IF NOT EXISTS receivers (
Receiver_ID INTEGER PRIMARY KEY,
Name TEXT,
Type TEXT,
City TEXT,
Contact TEXT
);
""")

In [112]:
# Create food_listings table with foreign key linking to providers
cur.execute("""
CREATE TABLE IF NOT EXISTS food_listings (
Food_ID INTEGER PRIMARY KEY,
Food_Name TEXT,
Quantity INTEGER,
Expiry_Date TEXT,
Provider_ID INTEGER,
Provider_Type TEXT,
Location TEXT,
Food_Type TEXT,
Meal_Type TEXT,
FOREIGN KEY(Provider_ID) REFERENCES providers(Provider_ID));
""")

In [113]:
# Create claims table with foreign keys to food_listings and receivers
cur.execute("""
CREATE TABLE IF NOT EXISTS claims (
Claim_ID INTEGER PRIMARY KEY,
Food_ID INTEGER,
Receiver_ID INTEGER,
Status TEXT,
Timestamp TEXT,
FOREIGN KEY(Food_ID) REFERENCES food_listings(Food_ID),
FOREIGN KEY(Receiver_ID) REFERENCES receivers(Receiver_ID)
);
""")

In [114]:
# 1. How many food providers and receivers are there in each city?
query = "select City, count(*) from providers group by City"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)
# query = "select City, count(*) 'Nos of Receivers' from receivers group by City"
# receivers_count = pd.read_sql_query(query, connection)
# print(receivers_count.head())
query = "select City, count(*) from receivers group by City"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

('West Omar', 1)
('Port Daniellechester', 1)
('Tyronebury', 1)
('New Bobbytown', 1)
('New William', 1)
('Charlesmouth', 1)
('Torresfort', 1)
('Kylehaven', 1)
('Johnsonborough', 1)
('Castilloport', 1)
('West Benjamin', 1)
('New Josemouth', 1)
('Port Leahfurt', 1)
('Lake Charleston', 1)
('Cisnerostown', 1)
('Lake Cheryl', 1)
('New Calebberg', 1)
('Owenschester', 1)
('Lake Lauraton', 1)
('Carrport', 1)
('Port Jacob', 1)
('Williamsland', 1)
('Lake Lisa', 1)
('North William', 1)
('Timothychester', 1)
('Tammyside', 2)
('Johnstonhaven', 1)
('East John', 1)
('West Lisamouth', 1)
('Steveport', 1)
('Port Anita', 1)
('Port John', 1)
('Jasonstad', 1)
('Lake Joseph', 1)
('Millerview', 1)
('South Joshua', 1)
('Port Victoria', 1)
('Kimberlychester', 1)
('Lake Christophermouth', 1)
('North Brooke', 1)
('Jamesfurt', 1)
('Penabury', 1)
('South Kevinhaven', 1)
('Karentown', 1)
('North Tracy', 1)
('East Lindsayville', 1)
('South Danielle', 1)
('New Aaronberg', 1)
('North Stephanieville', 1)
('North Gary',

In [115]:
# 2. Which type of food provider (restaurant, grocery store, etc.) contributes the most food?
query = "select Provider_Type, SUM(Quantity) from food_listings, providers where food_listings.Provider_ID = providers.Provider_ID group by Provider_Type order by SUM(Quantity) desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

('Restaurant', 6923)
('Supermarket', 6696)
('Catering Service', 6116)
('Grocery Store', 6059)


In [116]:
# # # 3. What is the contact information of food providers in a specific city?
query = "Select Name,City,Contact FROM providers where City = 'New Jessica';"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
  print (row)


('Gonzales-Cochran', 'New Jessica', '+1-600-220-0480')


In [117]:
# # # 3. What is the contact information of food providers in a specific city?
query = "SELECT City, Name, Contact FROM providers ORDER BY City;"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
  print (row)


('Adambury', 'Ibarra LLC', '6703380260')
('Adamsview', 'Lozano-Miller', '001-281-026-8022')
('Adamsville', 'Davis Ltd', '(112)122-3591x558')
('Aguirreville', 'Rowe-Chen', '8228891240')
('Alexanderchester', 'Galloway-Henderson', '001-867-928-0212x3211')
('Alexanderstad', 'Castillo, Nichols and Miller', '084-323-1485')
('Allenborough', 'Scott, Lopez and Baldwin', '001-590-644-2836')
('Allenton', 'Kelly, Cummings and Ward', '795.078.7850')
('Amandaborough', 'Baker, Valencia and Smith', '(397)967-0645')
('Amandashire', 'Avery PLC', '589.122.1214x989')
('Amberton', 'Morales, Wood and Cox', '468.025.0721x72227')
('Ambertown', 'Bell, Davis and Hickman', '+1-754-517-2435x37063')
('Amyport', 'Porter, Hernandez and Pace', '001-279-802-3400x771')
('Andersonmouth', 'Nguyen-Parsons', '(638)668-6852')
('Andersonville', 'Anderson, Robinson and Phillips', '7809531647')
('Andreaborough', 'Marshall Ltd', '+1-314-339-0357x53136')
('Andrewsmouth', 'Stewart-Soto', '(828)634-0022x45412')
('Andrewstad', 'Rus

In [118]:
# # # 3. What is the contact information of food providers in a specific city?
"SELECT Name, City, Contact FROM providers WHERE City IN (SELECT City FROM providers GROUP BY City HAVING COUNT(*) > 1) UNION ALL SELECT Name, City, Contact FROM providers WHERE City IN ( SELECT City FROM providers GROUP BY City HAVING COUNT(*) = 1)";
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

('Adambury', 'Ibarra LLC', '6703380260')
('Adamsview', 'Lozano-Miller', '001-281-026-8022')
('Adamsville', 'Davis Ltd', '(112)122-3591x558')
('Aguirreville', 'Rowe-Chen', '8228891240')
('Alexanderchester', 'Galloway-Henderson', '001-867-928-0212x3211')
('Alexanderstad', 'Castillo, Nichols and Miller', '084-323-1485')
('Allenborough', 'Scott, Lopez and Baldwin', '001-590-644-2836')
('Allenton', 'Kelly, Cummings and Ward', '795.078.7850')
('Amandaborough', 'Baker, Valencia and Smith', '(397)967-0645')
('Amandashire', 'Avery PLC', '589.122.1214x989')
('Amberton', 'Morales, Wood and Cox', '468.025.0721x72227')
('Ambertown', 'Bell, Davis and Hickman', '+1-754-517-2435x37063')
('Amyport', 'Porter, Hernandez and Pace', '001-279-802-3400x771')
('Andersonmouth', 'Nguyen-Parsons', '(638)668-6852')
('Andersonville', 'Anderson, Robinson and Phillips', '7809531647')
('Andreaborough', 'Marshall Ltd', '+1-314-339-0357x53136')
('Andrewsmouth', 'Stewart-Soto', '(828)634-0022x45412')
('Andrewstad', 'Rus

In [119]:
# 4. Which receivers have claimed the most food?
query = "select  c.Receiver_ID,type,Status,count(*) from receivers r,claims c where r.Receiver_ID= c.Receiver_ID and Status = 'Completed' group by c.Receiver_ID,type,Status having count(*) > 1"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

(747, 'Shelter', 'Completed', 2)
(440, 'Individual', 'Completed', 2)
(729, 'Charity', 'Completed', 2)
(905, 'Charity', 'Completed', 2)
(613, 'Shelter', 'Completed', 2)
(354, 'Charity', 'Completed', 2)
(407, 'Charity', 'Completed', 2)
(601, 'Shelter', 'Completed', 2)
(637, 'NGO', 'Completed', 2)
(538, 'Individual', 'Completed', 2)
(181, 'NGO', 'Completed', 2)
(418, 'Charity', 'Completed', 2)
(414, 'NGO', 'Completed', 2)
(781, 'NGO', 'Completed', 2)
(571, 'Shelter', 'Completed', 2)
(849, 'Shelter', 'Completed', 2)
(420, 'Charity', 'Completed', 2)
(168, 'Shelter', 'Completed', 2)
(850, 'Individual', 'Completed', 2)
(600, 'Individual', 'Completed', 2)
(482, 'NGO', 'Completed', 3)
(269, 'Individual', 'Completed', 2)
(505, 'NGO', 'Completed', 3)
(353, 'Individual', 'Completed', 2)
(423, 'Individual', 'Completed', 2)
(368, 'NGO', 'Completed', 2)
(306, 'Charity', 'Completed', 2)
(1, 'Shelter', 'Completed', 2)
(473, 'Shelter', 'Completed', 2)
(342, 'Charity', 'Completed', 2)
(183, 'Individual',

In [120]:
# 5. What is the total quantity of food available from all providers?
query = "select SUM(Quantity) from food_listings left join claims on claims.Food_ID = food_listings.Food_ID where Claim_ID is null"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

(9160,)


In [121]:
# 6. Which city has the highest number of food listings?

query = "select fl.provider_ID,city, SUM(Quantity) from providers P, food_listings FL where p.Provider_ID = FL.Provider_ID group by fl.Provider_ID,City order by SUM(Quantity) desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

(709, 'South Kathryn', 179)
(306, 'North Keith', 158)
(655, 'Jimmyberg', 150)
(315, 'Lake Andrewmouth', 142)
(678, 'Lake Travis', 140)
(499, 'Jonathanstad', 124)
(41, 'Bentleyburgh', 123)
(161, 'Perezport', 123)
(262, 'New Carol', 121)
(717, 'Pamelaberg', 116)
(146, 'Jonathanhaven', 116)
(498, 'East Craig', 115)
(7, 'Lake Christopherburgh', 114)
(139, 'Port Connie', 113)
(846, 'South Thomasville', 111)
(938, 'Hollyhaven', 110)
(995, 'Manningshire', 106)
(518, 'Anneville', 106)
(393, 'Heatherview', 104)
(752, 'East Heatherport', 104)
(478, 'Lake Regina', 102)
(251, 'East Terrancemouth', 101)
(192, 'Leslieville', 101)
(98, 'Gardnerfort', 99)
(332, 'West Whitneymouth', 99)
(485, 'East Andrea', 99)
(242, 'Michaelport', 98)
(47, 'East Andrewland', 98)
(312, 'North Ianbury', 97)
(346, 'Thomasville', 97)
(888, 'West Sharonview', 97)
(705, 'East Robert', 96)
(138, 'North Sherrimouth', 96)
(826, 'Lake Karenfurt', 96)
(892, 'Russellfurt', 96)
(4, 'Mendezmouth', 96)
(357, 'Donnaborough', 95)
(391

In [122]:
# 7. What are the most commonly available food types?

query = "SELECT Food_Type, COUNT(*) AS Availability_Count FROM Food_Listings GROUP BY Food_Type ORDER BY Availability_Count DESC"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

('Vegetarian', 336)
('Vegan', 334)
('Non-Vegetarian', 330)


In [123]:
# 8. How many food claims have been made for each food item?

query = "select f.food_name,count(Claim_ID) as claims  from providers p, food_listings f,claims c where p.Provider_ID = f.Provider_ID and f.Food_ID = c.Food_ID group by f.Food_Name order by claims  desc "
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

('Rice', 122)
('Soup', 114)
('Dairy', 110)
('Fish', 108)
('Salad', 106)
('Chicken', 102)
('Bread', 94)
('Pasta', 87)
('Vegetables', 86)
('Fruits', 71)


In [124]:
# 9. Which provider has had the highest number of successful food claims?

query = "select p.Provider_ID,count(*),Name from providers p, food_listings f,claims c where p.Provider_ID = f.Provider_ID and f.Food_ID = c.Food_ID group by p.Provider_ID,name having count(*) > 1 order by count(*) desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

(752, 12, 'Butler-Richardson')
(315, 9, 'Nelson LLC')
(384, 9, 'Rogers-Warren')
(709, 7, 'Barry Group')
(176, 7, 'Allen-Mccullough')
(241, 6, 'Harper, Blake and Alexander')
(993, 6, 'Mckinney Ltd')
(346, 6, 'Phillips, Wolfe and Martin')
(550, 6, 'Phelps, Graham and Hayes')
(108, 6, 'Williams-Cox')
(518, 5, 'Hill, Davis and Stewart')
(714, 5, 'Ramos Group')
(857, 5, 'Holmes-Schaefer')
(885, 5, 'Spears, Anderson and Huffman')
(215, 5, 'Simmons Ltd')
(677, 5, 'Gonzalez, Dunn and Oconnell')
(161, 5, 'Campbell LLC')
(237, 5, 'White, Crawford and Pearson')
(262, 5, 'Bradford-Martinez')
(999, 5, 'Moody Ltd')
(514, 5, 'Roy-Hancock')
(58, 5, 'Martin, Tyler and Wilkerson')
(616, 5, 'Moore-Flowers')
(850, 5, 'Cooper, Osborn and Clark')
(138, 5, 'Jones, Rojas and Brown')
(596, 5, 'Barnes, Castro and Curtis')
(251, 5, 'Carter-Jones')
(57, 4, 'Cook Group')
(135, 4, 'Dyer Group')
(308, 4, 'Doyle LLC')
(140, 4, 'Zhang LLC')
(413, 4, 'Davis, Bowen and Miller')
(312, 4, 'Garcia-Hunter')
(665, 4, 'Ortega

In [125]:
# 10. What percentage of food claims are completed vs. pending vs. canceled?

query = "SELECT status,COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) FROM claims GROUP BY status;"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)
# 11. What is the average quantity of food claimed per receiver?

query = "SELECT r.Name, AVG(quantity) AS Avg FROM claims c,food_listings f, receivers r where c.Food_ID =f.Food_ID and c.Receiver_ID = r.Receiver_ID GROUP BY r.Name order by Avg desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)


('Completed', Decimal('33.9000000000000000'))
('Cancelled', Decimal('33.6000000000000000'))
('Pending', Decimal('32.5000000000000000'))
('Peggy Knight', Decimal('50.0000000000000000'))
('Christopher Wright', Decimal('50.0000000000000000'))
('Nancy Silva', Decimal('50.0000000000000000'))
('Lisa Pitts', Decimal('50.0000000000000000'))
('Daniel Williams', Decimal('50.0000000000000000'))
('Nancy Jones', Decimal('50.0000000000000000'))
('Thomas Villanueva', Decimal('50.0000000000000000'))
('Scott Brown', Decimal('49.0000000000000000'))
('Jennifer Mccoy', Decimal('49.0000000000000000'))
('Timothy Patel DVM', Decimal('49.0000000000000000'))
('Alexander Mullins', Decimal('49.0000000000000000'))
('Christopher Gonzalez', Decimal('49.0000000000000000'))
('Angel Adams', Decimal('49.0000000000000000'))
('Melissa Little', Decimal('49.0000000000000000'))
('Cassandra Yoder', Decimal('49.0000000000000000'))
('Angela Wyatt', Decimal('49.0000000000000000'))
('Robert Stewart', Decimal('48.0000000000000000

In [126]:
# 11. What is the average quantity of food claimed per receiver?

query = "SELECT r.Name, AVG(quantity) AS Avg FROM claims c,food_listings f, receivers r where c.Food_ID =f.Food_ID and c.Receiver_ID = r.Receiver_ID GROUP BY r.Name order by Avg desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)


('Peggy Knight', Decimal('50.0000000000000000'))
('Christopher Wright', Decimal('50.0000000000000000'))
('Nancy Silva', Decimal('50.0000000000000000'))
('Lisa Pitts', Decimal('50.0000000000000000'))
('Daniel Williams', Decimal('50.0000000000000000'))
('Nancy Jones', Decimal('50.0000000000000000'))
('Thomas Villanueva', Decimal('50.0000000000000000'))
('Scott Brown', Decimal('49.0000000000000000'))
('Jennifer Mccoy', Decimal('49.0000000000000000'))
('Timothy Patel DVM', Decimal('49.0000000000000000'))
('Alexander Mullins', Decimal('49.0000000000000000'))
('Christopher Gonzalez', Decimal('49.0000000000000000'))
('Angel Adams', Decimal('49.0000000000000000'))
('Melissa Little', Decimal('49.0000000000000000'))
('Cassandra Yoder', Decimal('49.0000000000000000'))
('Angela Wyatt', Decimal('49.0000000000000000'))
('Robert Stewart', Decimal('48.0000000000000000'))
('Samantha Ellison', Decimal('48.0000000000000000'))
('Dustin Johnson', Decimal('48.0000000000000000'))
('Melissa Smith', Decimal('4

In [127]:
# 12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

query = "select Meal_Type, count(*) from  food_listings f,claims c where f.Food_ID = c.Food_ID group by Meal_Type having count(*) > 1 order by count(*) desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)


('Breakfast', 278)
('Lunch', 250)
('Snacks', 240)
('Dinner', 232)


In [128]:
# 13. What is the total quantity of food donated by each provider?

query = "SELECT p.Name,sum(quantity) as total from providers p, food_listings f where p.Provider_ID = f.Provider_ID group by p.Name order by total desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

('Miller Inc', 217)
('Barry Group', 179)
('Evans, Wright and Mitchell', 158)
('Smith Group', 150)
('Campbell LLC', 145)
('Nelson LLC', 142)
('Ruiz-Oneal', 140)
('Blankenship-Lewis', 124)
('Kelly-Ware', 123)
('Bradford-Martinez', 121)
('Hampton-Lee', 116)
('Shepherd and Sons', 116)
('Jones, Ortega and Rubio', 115)
('Ortiz-Lee', 114)
('Johnson-Ray', 113)
('Barker LLC', 110)
('Hill, Davis and Stewart', 106)
('Moore Group', 106)
('Butler-Richardson', 104)
('Steele Ltd', 104)
('Lopez, Roach and Roach', 102)
('Hunter, Ballard and Caldwell', 101)
('Carter-Jones', 101)
('Baker-Mcdonald', 99)
('Hogan-Johnston', 99)
('Cox LLC', 99)
('Jackson Ltd', 98)
('Wong-Reese', 98)
('Aguilar-Frederick', 98)
('Garcia-Hunter', 97)
('Phillips, Wolfe and Martin', 97)
('Schmidt-Alexander', 97)
('Flores-Wade', 96)
('Brown-Stephens', 96)
('Jones, Rojas and Brown', 96)
('Clark, Prince and Williams', 96)
('Rogers, Harmon and Gordon', 96)
('Carey PLC', 95)
('Lambert Ltd', 94)
('Wilson Group', 94)
('Doyle LLC', 93)
('

In [129]:
import streamlit as st
!pip install streamlit

DB_CONFIG = {
    "dbname" == "ramji",
    "username" == "postgres",
    "password" == "Ktm@9291",
    "host" == "localhost",
    "port" == 5432
}

connection = psycopg2.connect(host = "localhost",database = "ramji", user = "postgres" , password ="Ktm@9291")
cur = connection.cursor()

In [130]:
%%writefile Food_Waste_Project_finalDraft_app.py
import streamlit as st

# Set Streamlit page config
st.set_page_config(page_title="Local Food Wastage System", layout="wide")

# Page title
st.title("🍛 Local Food Wastage Management System")

# Sidebar navigation
menu = st.sidebar.radio("Select Section", ["Providers", "Receivers", "Food Listings", "Claims", "Analysis"])


Overwriting Food_Waste_Project_finalDraft_app.py


In [ ]:
!streamlit run Food_Waste_Project_finalDraft_app.py

In [ ]:

# Providers Section
menu = st.sidebar.radio("Select Section", ["Providers", "Receivers", "Food Listings", "Claims", "Analysis"])

if menu == "Providers":
    st.header("Food Providers by City")
    
    cities = pd.read_sql_query("SELECT DISTINCT City FROM providers_data ORDER BY City", conn)
    selected_city = st.selectbox("Select a City", cities['City'])

    query = "SELECT Name, Contact FROM providers_data WHERE City = %s"
    results = pd.read_sql_query(query, conn, params=(selected_city,))
    
    st.write(f"Found {len(results)} providers in {selected_city}:")
    st.dataframe(results)

In [ ]:
# Receivers Section
elif menu == "Receivers":
    st.header("🎯 Food Receivers by City")
    cities = pd.read_sql_query("SELECT DISTINCT City FROM receivers_data ORDER BY City", conn)
    selected_city = st.selectbox("Select a City", cities['City'])

    query = "SELECT Name, Contact FROM receivers_df WHERE City = ?"
    results = pd.read_sql_query(query, conn, params=(selected_city,))
    st.write(f"Found {len(results)} receivers in {selected_city}:")
    st.dataframe(results)


In [ ]:
# Food Listings Section
elif menu == "Food Listings":
    st.header("🍽 Available Food Listings")
    df = pd.read_sql_query("SELECT * FROM food_listings_data", conn)
    st.dataframe(df)


In [ ]:
# Claims Section
elif menu == "Claims":
    st.header("📦 Food Claims Overview")
    df = pd.read_sql_query("SELECT * FROM claims_data", conn)
    st.dataframe(df)


In [ ]:
# Analysis Section
elif menu == "Analysis":
    st.header("📊 Insights & SQL Queries")

    col1, col2 = st.columns(2)

    with col1:
        st.subheader("1. Providers & Receivers by City")
        df1 = pd.read_sql_query("""
            SELECT City, COUNT(*) AS Providers FROM providers_data GROUP BY City""", conn)
        df2 = pd.read_sql_query("""
            SELECT City, COUNT(*) AS Receivers FROM receivers_data GROUP BY City """, conn)
        merged = pd.merge(df1, df2, on="City", how="outer").fillna(0)
        st.dataframe(merged)

    with col2:
        st.subheader("2. Top Provider Types")
        df = pd.read_sql_query("""SELECT Type, COUNT(*) AS Total FROM providers_data GROUP BY Type ORDER BY Total DESC""", conn)
        st.dataframe(df)

    st.subheader("3. Completed vs. Pending vs. Cancelled Claims")
    df = pd.read_sql_query(""" SELECT Status, COUNT(*) AS Count FROM claims_data GROUP BY Status """, conn)
    st.bar_chart(df.set_index('Status'))

conn.close()

In [ ]:
!streamlit run Food_Waste_Project_finalDraft_app.py